<center><h1> Lab03 sheet </h1></center>

**Student ID: 22670287** <br>
**Student Name: Xiaoyan Huang**<br>
**Create Data: 1/4/2020**<br>
**End Data: 7/4/2020**<br>

Project1 is to train and evaluate the performance of a Voting classifier consisted of SVM, Logistic Regression and Gradient Descent classifier in the Cellular Localization Sites of Proteins on some E. Coli bacteria data.

# 1. Project1

## 1.1 Load data

In [ ]:
import pandas as pd

In [ ]:
colnames = ['Sequence Name','mcg','gvh','lip','chg','aac','alm1','alm2','class']

In [ ]:
eco = pd.read_csv("ecoli.data", header = None, sep = '\s+', names = colnames)

In [ ]:
eco.head()

In [ ]:
eco.shape

In [ ]:
eco['class'].value_counts()

In [ ]:
ecoli = eco.loc[~eco['class'].isin(['omL','imL','imS'])]

In [ ]:
ecoli['class'].value_counts()

In [ ]:
ecoli['class'].hist()

In [ ]:
ecoli.describe()

- From the result of value counts, it is not necessary to classify those classes having less than 10 instances. And classes ('omL','imL','imS') were removed.

- In addition, it is an **inbalanced** dataset that cytoplasm is dominant from the histogram. 

- At last, even if the range of numerical features above is between 0 and 1, SVM is one of classifier in the voting classifier that is sensitive to data normalization. Thus, there is no evidence to show it is not needed to normalization and I will make a contrast.

## 1.2 Split data into train and test dataset

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = ecoli.iloc[:,1:8]

In [ ]:
X.tail()

In [ ]:
y = ecoli['class']

In [ ]:
y.tail()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

- The dataset was performed an 80/20 split with 261 instances training set and 66 instances testing set.
- And *X_train_nm* and *X_test_nm* below were normalized, which rescales the values into a range of [0,1]

In [ ]:
from sklearn import preprocessing

In [ ]:
X_train_nm = preprocessing.normalize(X_train)

In [ ]:
X_train_nm

In [ ]:
X_test_nm = preprocessing.normalize(X_test)

In [ ]:
X_test_nm

## 1.3 Build a voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
log_clf = LogisticRegression(random_state=42, class_weight="balanced")
SGD_clf = SGDClassifier(random_state=42, class_weight="balanced")
svm_clf = svm.SVC(random_state=42, class_weight="balanced")

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('SGD', SGD_clf), ('svm', svm_clf)],
    voting='hard')

In [ ]:
voting_clf.fit(X_train, y_train)

In [ ]:
voting_clf_nm = VotingClassifier(
    estimators=[('lr', log_clf), ('SGD', SGD_clf), ('svm', svm_clf)],
    voting='hard')

In [ ]:
voting_clf_nm.fit(X_train_nm, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

In [ ]:
voting_clf.fit(X_train, y_train)
accuracy_score(y_test, voting_clf.predict(X_test))

In [ ]:
voting_clf_nm.fit(X_train_nm, y_train)
accuracy_score(y_test, voting_clf_nm.predict(X_test_nm))

- *voting_clf* consists of *log_clf*, *SGD_clf* and *svm_clf* with default hyperparameter values except the class_weight, as it is imbalanced dataset.
- In this experiment, *voting_clf_nm* also was created for comparison and the fact is that the dataset is **not needed to normalization**, if the features range between 0 and 1. The performance in *voting_clf* is better.

## 1.4 Build individual classifiers

In [ ]:
all_y_pred = []
all_score = []
all_f1 = []

In [ ]:
print("There are the accuracy for each classifier: \n")
for clf in (log_clf, SGD_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    all_y_pred.append(y_pred)
    score = accuracy_score(y_test, y_pred)
    all_score.append(score)
    f1 = metrics.f1_score(y_test, y_pred, average='macro')
    all_f1.append(f1)
    print(clf.__class__.__name__, score)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)

In [ ]:
print("The accuracy of LogisticRregression is %s"%all_score[0]) 
print("The f1-score of LogisticRregression is %s"%all_f1[0])
print("The confusion matrice of LogisticRregression")

conf_mx = confusion_matrix(y_test, all_y_pred[0])
plt.matshow(conf_mx, cmap=plt.cm.gray)

In [ ]:
print("The accuracy of SGDClassifier is %s"%all_score[1]) 
print("The f1-score of SGDClassifier is %s"%all_f1[1])
print("The confusion matrice of SGDClassifier")

confu_mx = confusion_matrix(y_test, all_y_pred[1])
plt.matshow(conf_mx, cmap=plt.cm.gray)

In [ ]:
print("The accuracy of svm is %s"%all_score[2]) 
print("The f1-score of svm is %s"%all_f1[2])
print("The confusion matrice of svm")

conf_mx = confusion_matrix(y_test, all_y_pred[2])
plt.matshow(conf_mx, cmap=plt.cm.gray)

In [ ]:
print("The accuracy of voting is %s"%all_score[3]) 
print("The f1-score of voting is %s"%all_f1[3])
print("The confusion matrice of voting")

conf_mx = confusion_matrix(y_test, all_y_pred[3])
plt.matshow(conf_mx, cmap=plt.cm.gray)

- Accuracy is a ratio of correctly predicted observation to the total observations.
- F1 score is the weighted average of Precision and Recall, F1 is usually more useful than accuracy if there is an uneven class distribution. After comparison, the voting classifier is better even though the differences between classifiers are small.